<div style="text-align: right"><i>Peter Norvig<br>Dec 2018<br>Updated Jun 2020</i></div>

# Portmantout Words

A [***portmanteau***](https://en.wikipedia.org/wiki/Portmanteau) is a word that squishes together two words, like  *math* + *athlete* = *mathlete*.  Inspired by [**Darius Bacon**](http://wry.me/), I covered this as a programming exercise in my 2012 [**Udacity course**](https://www.udacity.com/course/design-of-computer-programs--cs212). In 2018 I was re-inspired by [**Tom Murphy VII**](http://tom7.org/), who added a new twist:  [***portmantout words***](http://www.cs.cmu.edu/~tom7/portmantout/) (*tout* from the French for *all*), which are defined as:

> A **portmantout** of a set of words $W$ is a string $S$ such that:
* Every word in $W$ is a **substring** of $S$.
* The words **overlap**: every word (except the first) starts at an index that is equal to or before the end of another word.
* **Nothing else** is in $S$: every letter in $S$ comes from the overlapping words. (But a word may be repeated any number of times.)

Although not part of the definition, the goal is to get as short an $S$ as possible, and to do it for a set of about 100,000 words. Developing a program to do that is the goal of this notebook.  My program (also available as [`portman.py`](portman.py)) helped me discover:

- **preferendumdums**: a political commentary portmanteau of {prefer, referendum, dumdums}
- **fortyphonshore**: a dire weather report portmanteau of {forty, typhons, onshore}; 
- **allegestionstage**: a brutal  theatre critic portmanteau of {alleges, egestions, onstage}. 
- **skymanipulablearsplittingler**: a nerve-damaging aviator portmanteau of {skyman, manipulable, blears, earsplitting, tinglers}
- **edinburgherselflesslylyricize**: a Scottish music review portmanteau of {edinburgh, burghers, herself, selflessly, slyly, lyricize}
- **impromptutankhamenability**: a portmanteau of {impromptu, tutankhamen, amenability}, indicating a willingness to see the Egyptian exhibit on the spur of the moment.
- **dashikimonogrammarianarchy**: a portmanteau of {dashiki, kimono, monogram, grammarian, anarchy}, describing the chaos that ensues when a linguist gets involved in choosing how to enscribe African/Japanese garb. 

# Problem-Solving Strategy

Although I haven't proved it, my intuition is that finding a shortest $S$ is an NP-hard problem, and with 100,000 words to cover, it is unlikely that I can find an optimal solution in a reasonable amount of time. A common approach to NP-hard problems is a **greedy algorithm**: make the locally best choice at each step, in the hope that the steps will fit together into a solution that is not too far from the globally best solution. 

Thus, my approach will be to build up a **path**, starting with one word, and then adding **steps**  to the end of the evolving path, one at a time. Each step consists of a word from *W* that overlaps the end of the previous word by at least one letter. I will choose the step that seems to be the best choice at the time (the one with the **lowest cost**), and will never undo a step, even if the path seems to get stuck later on. I distinguish two types of steps:

- **Unused word step**: using a word for the first time. (Once we use them all, we're done.)
- **Bridging word step**: if no remaining unused word overlaps the previous word, we need to do something to get back on track to consuming unused words. I call that something a **bridge**: a step that **repeats** a previously-used word and has a word ending that matches the start of some unused word. Repeating a word may add letters to the final length of $S$, and it doesn't get us closer to the requirement of using all the words, but it is sometimes necessary. Sometimes two words are required to build a bridge, but never more than two (at least with the 100,000 word set we will be dealing with). 

There's actually a third type of word, but it doesn't need a corresponding type of step: a **subword** is a word that is a substring of another word. If, say, `ajar` is in $W$, then we know we have to place it in some step along the path. But if `jar` is also in $W$, we don't need a step for it&mdash;whenever we place `ajar`, we have automatically placed `jar`. The program can save computation time save time by initializing the set of **unused words** to be the **nonsubwords** in $W$.  (We are still free to use a subword as a **bridging word** if needed.)

(*English trivia:* I use the clumsy term "nonsubwords" rather than "superwords", because there are a small number of words, like "cozy" and "pugs" and "july," that are not subwords of any other words but are also not superwords: they have no subwords.)

If we're going to be **greedy**, we need to know what we're being greedy about. I will always choose a step that minimizes the **number of excess letters** that the step adds. To be precise: I arbitrarily assume a baseline model in which all the words are concatenated with no repeated words and no overlap between them. (That's not a valid solution, but it is useful as a benchmark.) So if I add an unused word, and overlap it with the previous word by three letters, that is an excess of -3 (a negative excess is a positive thing): I've saved three letters over just concatenating the unused word. For a bridging word, the excess is the number of letters that do not overlap either the previous word or the next word. 

**Examples:** In each row of the table below, `'ajar'` is the previous word,  but each row makes different assumptions about what unused words remain, and thus we get different choices for the step to take. The table shows the overlapping letters between the previous word and the step, and in the case of bridges, it shows the next unused word that the step is bridging to. In the final two columns we give the excess score and excess letters.

|Previous|Step(s)|Overlap|Bridge to|Type of Step|Excess|Letters|
|--------|----|----|---|---|---|---|
| ajar|jarring|jar||unused word |-3| |
| ajar|arbitrary|ar||unused word |-2||
| ajar|argot|ar|goths|one-step bridge |0||
| ajar|arrow|ar|owlets| one-step bridge|1|r|
| ajar|rani+iraq|r|quizzed| two-step bridge|5 |anira | 

Let's go over the example steps one at a time:
- **jarring**: Here we assume `jarring` is an unused word, and it overlaps by 3 letters, giving it an excess cost of -3, which is the best possible (an overlap of 4 would mean `ajar` is a subword, and we already agreed to eliminate subwords).
- **arbitrary**: an unused word that overlaps by 2 letters, so it would only be chosen if there were no unused words with 3-letter overlap.
- **argot**: From here on down, we assume there are no unused words that overlap, which means that `argot` is a best-possible bridge, because it completely overlaps the `ar` in `ajar` and the `got` in an unused word, `goths`.
- **arrow**: overlaps the `ar` in `ajar` and the `ow` in an unused word, `owlets`, but that leaves one excess letter, `r`, for an excess cost of 1.
- **rani + iraq**: Suppose `quizzed` is the only remaining unused word. There is no single word that bridges from any suffix of `ajar` to any prefix of `quizzed`. But I have arranged to build two-word bridges for every combination of one letter on the left and one letter on the right (unless there was already a shorter one-word bridge for that combination). The bridge from `'r'` to `'q'` is  `rani` followed by `iraq`, which has an excess score of 5 due to the letters `anira` not overlapping anything.

We see that unused word steps always have a negative excess cost (that's good) while bridge steps always have a zero or positive excess cost; thus an unused word step is always better than a bridge step (according to this metric).




# Data Type Implementation

Here I describe how to implement the main data types in Python:

- **Word**: a Python `str` (as are subparts of words, like suffixes or individual letters).
- **Wordset**: a subclass of `set`, denoting a set of words.
- **Path**: a Python `list` of steps.
- **Step**: a tuple of `(overlap, word)`. The step adding `jarring` to `ajar` would be `(3, 'jarring')`, indicating an overlap of three letters. The first step in a path should have an overlap of 0; all others should have a positive integer overlap.
- **Bridge**: a tuple of an excess cost followed by one or two steps, e.g. `(1, (2, 'arrow'))`.
- **Bridges**: a precomputed and cached table mapping a prefixes and suffix to a bridge. For example:

      W.bridges['ar']['ow'] == (1, (2, 'arrow'))
      W.bridges['r']['q']  == (5, (1, 'rani'), (1, 'iraq'))

(*Python trivia:* I implemented `Wordset` as a subclass of `set` so that I can add attributes like `W.bridges`. You can do that with a user-defined subclass, but not with a builtin class.)

In the following code, data tyes are **Capitalized** and indexes into tuples are **UPPERCASE**.

In [1]:
from collections import defaultdict, Counter
from typing import List, Tuple, Set, Dict, Any

Word = str
class Wordset(set): """A set of words."""
Step = Tuple[int, str] # An (overlap, word) pair.
OVERLAP, WORD = 0, 1 # Indexes of the two parts of a Step.
Path = List[Step] # A list of steps.
Bridge = (int, Step,...) # An excess letter count and step(s), e.g. (1, (2, 'arrow')).
EXCESS, STEPS = 0, slice(1, None) # Indexes of the two parts of a bridge.

# 108,709 Word Set 

We can make Tom Murphy's 108,709-word list `"wordlist.asc"` into a `Wordset`,  $W$:

In [2]:
! [ -e wordlist.asc ] || curl -O https://norvig.com/ngrams/wordlist.asc

W = Wordset(open('wordlist.asc').read().split()) 

# Overall Program Design

I  thought I would define a major function, `portman`, to generate the portmantout string $S$ from the set of words $W$ according to the strategy outlined above, and a minor function, `is_portman`, to verify the result. But I found verification to be difficult. For example, given  $S =$  `'...helloworld...'` I would reject that as non-overlapping if I parsed it as `'hello'` + `'world'`, but I would accept it if parsed as `'hell'` + `'low'` + `'world'`. It was hard for `is_portman` to decide which parse was intended, which is a shame because `portman` *knew* which was intended, but  discarded the information. 

Therefore, I decided to change the interface: I'll have one function that takes $W$ as input and returns  a path $P$, and a second function to generate the string $S$ from $P$.  I decided on the following calling and [naming](https://en.wikipedia.org/wiki/Natalie_Portman) conventions:

    P = natalie(W)    # Find a portmantout path P for a set of words W
    is_portman(P, W)  # Verify whether P is a valid path covering words W
    S = portman(P)    # Compute the string S from the path P

Thus I can generate a string $S$ with:

    S = portman(natalie(W))
    
# portman

Here is the definition of `portman` and the result of `portman(P1)`, covering the word set `W1` (which has five superwords and 16 subwords):

In [3]:
def portman(P: Path) -> Word:
    """Compute the portmantout string S from the path P."""
    return ''.join(word[overlap:] for (overlap, word) in P)

In [4]:
W1 = Wordset(('anarchy', 'dashiki', 'grammarian', 'kimono', 'monogram',
              'an', 'am', 'arc', 'arch', 'aria', 'as', 'ash', 'dash', 
              'gram', 'grammar', 'i', 'mar', 'narc', 'no', 'on', 'ram'))
P1 = [(0, 'dashiki'),
      (2,      'kimono'),
      (4,        'monogram'),
      (4,            'grammarian'),
      (2 ,                   'anarchy')]
S1 =      'dashikimonogrammarianarchy'

In [5]:
portman(P1)

'dashikimonogrammarianarchy'

# natalie

As stated above, the approach is to start with one word (either given as an optional argument to `natalie` or chosen arbitrarily from the word set $W$), and then repeatedly add steps, each step being either an unused word or a bridging word. In order to make this process efficient, we arrange for `precompute(W)` to cache the following information in `W`:
  - `W.subwords`: a set of all the words that are contained within another word in `W`.
  - `W.bridges`: a dict where `W.bridges[suf][pre]` gives the best bridge between the two word affixes.
  - `W.unused`: initially the set of nonsubwords in `W`; when a word is used it is removed from the set.
  - `W.startswith`: a dict that maps from a prefix to all the unused words that start with the prefix. A word is removed from all the places it appears when it is used. Example: `W.startswith['somet'] == {'something', 'sometimes'}`.
  
These structures are somewhat complicated, so don't be discouraged if you have to go over a line of code or a prose description word-by-word several times before you understand exactly how it works.

After the precomputation, `natalie` loops until there are no more unused words. On each turn we call `unused_step`, which returns a list of one step if an unused word overlaps, or the empty list if it doesn't, in which case we call `bridging_steps`, which always returns a bridge of either one or two steps. We then append the step(s) to the path `P`, and call `used(W, word)` to mark that `word` has been used (reducing the size of `W.unused` and updating `W.startswith` if `word` was previously unused). 

It is important that every bridge leads to an unused word. That way we know the program will **always terminate**: if  $N$ is the number of unused nonsubwords in $W$, then consider the quantity $(2N + (1$ `if` the last step overlaps an unused word `else` $0))$. Every iteration of the `while` loop decreases this quantity by at least 1; therefore the quantity will eventually be zero, and when it is zero, it must be that `W.unused` is empty and the loop terminates.

In [6]:
def natalie(W: Wordset, start=None) -> Path:
    """Return a portmantout path containing all words in W."""
    precompute(W)
    word = start or first(W.unused)
    used(W, word)
    P = [(0, word)]
    while W.unused:
        steps = unused_step(W, word) or bridging_steps(W, word)
        for (overlap, word) in steps:
            P.append((overlap, word))
            used(W, word)
    return P

`unused_steps` considers every suffix of the previous word, longest suffix first. If a suffix starts any unused words, we choose the first such word as the step. Since we're going longest-suffix first, no other word could do better, and since unused steps have negative costs and bridges don't, the unused step will always be better than any bridge.

`bridging_steps` also tries every suffix of the previous word, and for each one it looks in the `W.bridges[suf]` table to see what prefixes (of unused words) we can bridge to from this suffix. Consider all such `W.bridges[suf][pre]` entries that bridge to the prefix of an unused word (as maintained in `W.startswith[pre]`). Out of all such bridges, take one with the minimal excess cost, and return the one- or two-step sequence that makes up the bridge.

So, both `unused_steps` and `bridging_steps` return a list of steps, the former either zero or one steps; the latter either one or two steps.

In [7]:
def unused_step(W: Wordset, prev_word: Word) -> List[Step]:
    """Return [(overlap, unused_word)] or []."""
    for suf in suffixes(prev_word):
        for unused_word in W.startswith.get(suf, ()):
            overlap = len(suf)
            return [(overlap, unused_word)]
    return []

def bridging_steps(W: Wordset, prev_word: Word) -> List[Step]:
    """The steps from the shortest bridge that bridges 
    from a suffix of prev_word to a prefix of an unused word."""
    bridge = min(W.bridges[suf][pre] 
                 for suf in suffixes(prev_word) if suf in W.bridges
                 for pre in W.bridges[suf] if W.startswith[pre])
    return bridge[STEPS]

(*Python trivia:* in `unused_step` I do `W.startswith.get(suf, ())`, not `W.startswith[suf]`  because the dict in question is a `defaultdict(set)`, and if there is no entry there, I don't want to insert an empty set entry.)

**Failure is not an option**: what happens if we have a small word set that can't make a portmantout? First `unused_step` will fail to find an unused word, which is fine; then `bridging_steps` will fail to find a bridge, and will raise `ValueError: min() arg is an empty sequence`. You could catch that error and return, say, an empty path if you wanted to, but my intended use is for word sets where this can never happen.

# precompute etc.

Here are a bunch of the subfunctions that make the code above work:

In [8]:
def precompute(W):
    """Precompute and cache data structures for W. The .subwords and .bridges
    data structures are static and only need to be computed once; .unused and
    .startswith are dynamic and must be recomputed on each call to `natalie`."""
    if not hasattr(W, 'subwords') or not hasattr(W, 'bridges'): 
        W.subwords = subwords(W)
        W.bridges  = build_bridges(W)
    W.unused       = W - W.subwords
    W.startswith   = compute_startswith(W.unused)
    
def used(W, word):
    """Remove word from `W.unused` and, for each prefix, from `W.startswith[pre]`."""
    assert word in W, f'used "{word}", which is not in the word set'
    if word in W.unused:
        W.unused.remove(word)
        for pre in prefixes(word):
            W.startswith[pre].remove(word)
            if not W.startswith[pre]:
                del W.startswith[pre]
                
def first(iterable, default=None): return next(iter(iterable), default)

def multimap(pairs) -> Dict[Any, set]:
    """Given (key, val) pairs, make a dict of {key: {val,...}}."""
    result = defaultdict(set)
    for key, val in pairs:
        result[key].add(val)
    return result

def compute_startswith(words) -> Dict[str, Set[Word]]: 
    """A dict mapping a prefix to all the words it starts:
    {'somet': {'something', 'sometimes'},...}."""
    return multimap((pre, w) for w in words for pre in prefixes(w))

def subwords(W: Wordset) -> Set[str]:
    """All the words in W that are subparts of some other word."""
    return {subword for w in W for subword in subparts(w) & W}              
    
def suffixes(word) -> List[str]:
    """All non-empty proper suffixes of word, longest first."""
    return [word[i:] for i in range(1, len(word))]

def prefixes(word) -> List[str]:
    """All non-empty proper prefixes of word."""
    return [word[:i] for i in range(1, len(word))]

def subparts(word) -> Set[str]:
    """All non-empty proper substrings of word"""
    return {word[i:j] 
            for i in range(len(word)) 
            for j in range(i + 1, len(word) + (i > 0))}

(*Python trivia:* the `(i > 0)` in the last line of  `subparts` means that for a four-letter word like `'gram'`, we include subparts `word[i:4]` except when `i == 0`, thus including `'ram'`, `'am'`, and `'m'`, but not `'gram'`. In Python it is considered good style to have a Boolean expression like `(i > 0)` automatically converted to an integer `0` or `1`.)

(*Math trivia:* "Proper" means "not whole". A proper subset is a subset that is not the whole set itself; a proper subpart of a word is a part (i.e. substring) of the word that is not the whole word itself.)

# Building Bridges

The last piece of the program is the construction of the `W.bridges` table. Recall that we want `W.bridges[suf][pre]` to be a bridge between a suffix of the previous word and a prefix of an unused word, as in the examples:

      W.bridges['ar']['ow'] == (1, (2, 'arrow'))
      W.bridges['ar']['c']  == (0, (2, 'arc'))
      W.bridges['r']['q']   == (5, (1, 'rani'), (1, 'iraq'))
      
We build all the bridges once and for all in `precompute`, and don't update them as words are used. Thus, `W.bridges['r']['q']` says "if there are any unused words starting with `'q'`, you can use this bridge, but I'm not promising there are any." The caller (i.e. `bridging_steps`) is responsible for checking that `W.startswith['q']` contains unused word(s).
      
Bridges should be short. We don't need to consider `antidisestablishmentarianism` as a possible bridge word. Instead, from our 108,709  word set $W$, we'll select the 10,273 words with length up to 5, plus 20 six-letter words that end in any of 'qujvz', the rarest letters. (For other word sets, you may have to tune these parameters.) I call these `shortwords`. I also compute a `shortstartswith` table for the `shortwords`, where, for example,

     shortstartswith['som'] == {'soma', 'somas', 'some'} # but not 'somebodies', 'somersaulting', ...
     
To build one-word bridges, consider every shortword, and split it up in all possible ways into a prefix that will overlap the previous word, a suffix that will overlap the next word, and a count of zero or more excess letters in the middle that don't overlap anything. For example:

In [9]:
def splits(word) -> List[Tuple[int, str, str]]: 
    """A sequence of (excess, pre, suf) tuples."""
    return [(excess, word[:i], word[i+excess:])
            for excess in range(len(word) - 1)
            for i in range(1, len(word) - excess)]

In [10]:
splits('arrow')

[(0, 'a', 'rrow'),
 (0, 'ar', 'row'),
 (0, 'arr', 'ow'),
 (0, 'arro', 'w'),
 (1, 'a', 'row'),
 (1, 'ar', 'ow'),
 (1, 'arr', 'w'),
 (2, 'a', 'ow'),
 (2, 'ar', 'w'),
 (3, 'a', 'w')]

The first element of the list says that `'arrow'` can bridge from `'a'` to `'rrow'` with 0 excess letters; the last says it can bridge from `'a'` to `'w'` with 3 excess letters (which happen to be `'rro'`). We consider every possible split, and pass it on to `try_bridge`, which records the bridge in the table under `bridges[pre][suf]` unless there is already a shorter bridge stored there.

In [11]:
def try_bridge(bridges, pre, suf, excess, word, step2=None):
    """Store a new bridge if it has less excess than the previous bridges[pre][suf]."""
    if suf not in bridges[pre] or excess < bridges[pre][suf][EXCESS]:
        bridge = (excess, (len(pre), word))
        if step2: bridge +=  (step2,)
        bridges[pre][suf] = bridge

Now for two-word bridges. I thought that if I allowed all possible two-word bridges the program would be slow because there would be so many of them, and most of them would be too long to be of any use. Thus, I decided to only use two-word bridges that bridge from the last letter in the previous word to the first letter in an unused word.

We start out the same way, looking at every shortword. But this time we look at every suffix of each shortword, and see if the suffix starts another shortword. If it does, then we have a two-word bridge. Here's the complete `build_bridges` function:

In [12]:
def build_bridges(W: Wordset, maxlen=5, end='qujvz'):
    """A table of bridges[pre][suf] == (excess, (overlap, word)), e.g.
    bridges['ar']['c'] == (0, (2, 'arc'))."""
    bridges         = defaultdict(dict)
    shortwords      = [w for w in W if len(w) <= maxlen + (w[-1] in end)]
    shortstartswith = compute_startswith(shortwords)
    # One-word bridges
    for word in shortwords: 
        for excess, pre, suf, in splits(word):
            try_bridge(bridges, pre, suf, excess, word)
    # Two-word bridges
    for word1 in shortwords:
        for suf in suffixes(word1): 
            for word2 in shortstartswith[suf]: 
                excess = len(word1) + len(word2) - len(suf) - 2
                A, B = word1[0], word2[-1]
                if A != B:
                    step2 = (len(suf), word2)
                    try_bridge(bridges, A, B, excess, word1, step2)
    return bridges

(*Note:* When we are *using* a bridge, we say `W.bridges[suf][pre]` but when we are *building* the bridge we say `W.bridges[pre][suf]`. It may seem confusing to write the code both ways, but that's because the very definition of **overlapping** is that the suffix of one word is the same as the prefix of the next, so it just depends how you are looking at it.)

**Missing bridges:** How do we know if a word set will always generate a portmantout? If the word set has a bridge from every one-letter suffix to every one other one-letter prefix, then it will always be able to bridge from any word to any unused word.  (Of course, there are other conditions under which it can succeed.) The function `missing_bridges` tells us which of these one-letter-to-one-letter bridges are missing:

In [13]:
def missing_bridges(W):
    """What one-letter-to-one-letter bridges are missing from W.bridges?"""
    return {A + B for A in alphabet for B in alphabet 
            if A != B and B not in W.bridges[A]}

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [14]:
precompute(W)
missing_bridges(W)

set()

Great! $W$ has no missing bridges. But the tiny word set $W1$ is missing 630 out of a possible 650 bridges:

In [15]:
precompute(W1)
len(missing_bridges(W1)), 26 * 25

(630, 650)

# Portmantout Solutions

**Finally!** We're ready to make portmantouts. First for the small word list `W1`:

In [16]:
natalie(W1, start='dashiki')

[(0, 'dashiki'),
 (2, 'kimono'),
 (4, 'monogram'),
 (4, 'grammarian'),
 (2, 'anarchy')]

In [17]:
portman(natalie(W1, start='dashiki'))

'dashikimonogrammarianarchy'

Let's make the portmantout and see how many steps and how many letters it is, and how long it takes:

In [18]:
%time P = natalie(W)
S = portman(P)
len(P), len(S)

CPU times: user 9.1 s, sys: 151 ms, total: 9.25 s
Wall time: 9.56 s


(103470, 553747)

We can specify the starting word, as Tom Murphy did:

In [19]:
%time P = natalie(W, start='portmanteau')
S = portman(P)
len(P), len(S)

CPU times: user 8.79 s, sys: 103 ms, total: 8.89 s
Wall time: 9.06 s


(103466, 553742)

I thought it might take 10 minutes, so under 10 seconds is super. 

# Making it Prettier

Notice I haven't actually *looked* at the portmantout yet. I didn't want to dump half a million letters into an output cell. Instead, I'll define `report` to summarize the results and save the full string $S$ into [a file](natalie.txt). I introduce the notion of **safe bridges**, which means that `W.bridges` contains a bridge from every leter to every other letter. A word set that has that property can always find a solution; word sets that don't might or might not find a solution, depending on the order of step choices.

In [20]:
def report(W, P, steps=50, letters=500, save='natalie.txt'):
    S       = portman(P)
    sub     = W.subwords
    nonsub  = W - sub
    bridge  = len(P) - len(nonsub)
    missing = len(missing_bridges(W)) or "no"
    valid   = "is" if is_portman(P, W) else "IS NOT"
    def L(words): return sum(map(len, words)) # Number of letters
    print(f'W has {len(W):,d} words ({len(nonsub):,d} nonsubwords; {len(sub):,d} subwords).')
    print(f'P has {len(P):,d} steps ({len(nonsub):,d} nonsubwords; {bridge:,d} bridge words).')
    print(f'S has {len(S):,d} letters; W has {L(W):,d}; nonsubs have {L(nonsub):,d}.')
    print(f'P has an average overlap of {(L(w for _,w in P)-len(S))/(len(P)-1):.2f} letters.')
    print(f'S has a compression ratio (letters(W)/letters(S)) of {L(W)/len(S):.2f}.')
    print(f'P (and thus S) {valid} a valid portmantout of W.')
    print(f'W has {missing} missing one-letter-to-one-letter bridges.')
    if save:
        print(f'S saved as "{save}", {open(save, "w").write(S)} bytes.')
    print(f'\nThe first and last {steps} steps are:\n')
    for step in [*P[:steps], '... ...', *P[-steps:]]:
        print(step)
    print(f'\nThe first and last {letters} letters are:\n\n{S[:letters]} ... {S[-letters:]}')

def is_portman(P: Path, W: Wordset) -> str:
    """Verify that P forms a valid portmantout string for W."""
    all_words = (W - W.subwords) <= set(w for (_, w) in P) <= W
    overlaps  = all(overlap > 0 and P[i - 1][1][-overlap:] == word[:overlap]
                    for i, (overlap, word) in enumerate(P[1:], 1))
    return all_words and overlaps and P[0][OVERLAP] == 0 # first step has 0 overlap

(*Python trivia:* if `X, Y` and `Z` are sets, `X <= Y <= Z` means "is `X` a subset of `Y` and `Y` a subset of `Z`?" We use the notation here to say that the set of words in $P$ must contain all the nonsubwords and can only contain words from $W$.)

In [21]:
report(W, P)

W has 108,709 words (64,389 nonsubwords; 44,320 subwords).
P has 103,466 steps (64,389 nonsubwords; 39,077 bridge words).
S has 553,742 letters; W has 931,823; nonsubs have 595,805.
P has an average overlap of 1.65 letters.
S has a compression ratio (letters(W)/letters(S)) of 1.68.
P (and thus S) is a valid portmantout of W.
W has no missing one-letter-to-one-letter bridges.
S saved as "natalie.txt", 553742 bytes.

The first and last 50 steps are:

(0, 'portmanteau')
(2, 'autographic')
(7, 'graphicness')
(3, 'essayists')
(2, 'tsarisms')
(1, 'skydiving')
(3, 'ingenuously')
(3, 'slyest')
(4, 'yesterdays')
(4, 'daysides')
(5, 'sideswiping')
(4, 'pingrasses')
(5, 'assessee')
(3, 'seeking')
(4, 'kinged')
(2, 'editresses')
(3, 'sestets')
(5, 'stetsons')
(4, 'sonships')
(5, 'shipshape')
(5, 'shapelessness')
(3, 'essayers')
(3, 'erstwhile')
(5, 'whiles')
(3, 'lessening')
(3, 'ingested')
(4, 'stedhorses')
(6, 'horseshoes')
(5, 'shoestrings')
(5, 'ringsides')
(5, 'sideslipping')
(3, 'ingrafting'

# Exploring

The program is complete, but there are still many interesting things to explore. 

**My first question**: is there an imbalance in starting and ending letters of words? That could lead to a need for many two-word bridges. We saw that the last 50 steps of $P$ all involved words that start with `q`, or bridges to them. 

In [22]:
precompute(W)

In [23]:
Counter(w[0] for w in W.unused).most_common() # How many nonsubwords start with each letter?

[('s', 7388),
 ('c', 5849),
 ('p', 4977),
 ('d', 4093),
 ('r', 3811),
 ('b', 3776),
 ('a', 3528),
 ('m', 3405),
 ('t', 3097),
 ('f', 2794),
 ('i', 2771),
 ('u', 2557),
 ('e', 2470),
 ('g', 2177),
 ('h', 2169),
 ('o', 1797),
 ('l', 1634),
 ('w', 1561),
 ('n', 1542),
 ('v', 1032),
 ('j', 638),
 ('k', 566),
 ('q', 330),
 ('y', 207),
 ('z', 169),
 ('x', 51)]

In [24]:
Counter(w[-1] for w in W.unused).most_common() # How many nonsubwords end with each letter?

[('s', 29056),
 ('y', 8086),
 ('d', 7520),
 ('g', 6343),
 ('e', 3215),
 ('t', 2107),
 ('r', 1994),
 ('n', 1860),
 ('l', 1182),
 ('c', 908),
 ('m', 657),
 ('a', 384),
 ('h', 351),
 ('k', 157),
 ('i', 128),
 ('p', 123),
 ('o', 113),
 ('x', 68),
 ('f', 51),
 ('w', 42),
 ('z', 21),
 ('u', 11),
 ('v', 6),
 ('b', 6)]

Yes, there is a problem: there are 330 words that start with `q` and no nonsubwords that end in `q` (there are two subwords, `colloq` and `iraq`). There is also a problem with 29,056 nonsubwords ending in `s` and only 7,388 starting with `s`. But many words start with combinations like `as` or `es` or `ps`, whereas there are few chances for a `q` at the start of a word to match up with a `q` near the end of a word.

Here are all the words that have a `q` as one of their last three letters:

In [25]:
q3 = {w for w in W if 'q' in w[-3:]}
' '.join(q3), len(q3)

('diplomatique faqir mystique relique mosque arabesque macaque maqui mozambique catafalque plaque brusque bisque unique obloquy manque perique applique claque boutique iraq grotesque cheque picaresque statuesque oblique opaque marque toque basque cinque obsequy aqua iraqis barque cinematheque critique odalisque albuquerque prosequi colloq tuque pulque pratique remarque baroque colloquy iraqi soliloquy technique burlesque placque discotheque hdqrs pique antique bosque cliquy semiopaque picturesque cirque romanesque torque yanqui ventriloquy casque clique physique masque bezique communique risque',
 72)

**My second question**: what are the most common steps in $P$? These will be bridge words. What do they have in common?

In [26]:
Counter(P).most_common(25)

[((1, 'so'), 2561),
 ((1, 'sap'), 2536),
 ((1, 'dab'), 2360),
 ((1, 'sic'), 2223),
 ((1, 'of'), 2039),
 ((2, 'lyre'), 1643),
 ((1, 'sun'), 1519),
 ((1, 'sin'), 1400),
 ((1, 'yam'), 867),
 ((2, 'lye'), 734),
 ((1, 'go'), 679),
 ((1, 'yow'), 612),
 ((1, 'spa'), 610),
 ((1, 'econ'), 609),
 ((1, 'gem'), 562),
 ((1, 'gun'), 487),
 ((1, 'yen'), 465),
 ((3, 'erst'), 454),
 ((2, 'type'), 447),
 ((1, 'she'), 390),
 ((1, 'you'), 371),
 ((1, 'sex'), 324),
 ((1, 'simp'), 317),
 ((1, 'tv'), 312),
 ((1, 'gal'), 297)]

Even though `iraq` dominated the last 50 steps, that's not true of `P` overall. Instead, it looks like bridging away from `s` is a big concern (as expected by the letter counts). (Also, `lyre` and `lye` bridge from an adverb ending.) The following says that about 30% of all steps in `P` bridge from a suffix that contains `s`:

In [27]:
sum('s' in w[:o] for o, w in P) / len(P)

0.2971507548373379

**My third question:** What is the distribution of word lengths? What is the longest word? What is the distribution of letters?

In [28]:
Counter(sorted(map(len, W.unused))) # Counter of word lengths

Counter({3: 2,
         4: 186,
         5: 1796,
         6: 4364,
         7: 8672,
         8: 11964,
         9: 11950,
         10: 8443,
         11: 6093,
         12: 4423,
         13: 2885,
         14: 1765,
         15: 1017,
         16: 469,
         17: 198,
         18: 91,
         19: 33,
         20: 22,
         21: 9,
         22: 4,
         23: 2,
         28: 1})

In [29]:
max(W, key=len)

'antidisestablishmentarianism'

In [30]:
Counter(L for w in W.unused for L in w).most_common() # Counter of letters

[('e', 68038),
 ('s', 60080),
 ('i', 53340),
 ('a', 43177),
 ('n', 42145),
 ('r', 41794),
 ('t', 38093),
 ('o', 35027),
 ('l', 32356),
 ('c', 23100),
 ('d', 22448),
 ('u', 19898),
 ('g', 17815),
 ('p', 16128),
 ('m', 16062),
 ('h', 12673),
 ('y', 11889),
 ('b', 11581),
 ('f', 7885),
 ('v', 5982),
 ('k', 4892),
 ('w', 4880),
 ('z', 2703),
 ('x', 1677),
 ('j', 1076),
 ('q', 1066)]

**My fourth question**: How many bridges are there? How many excess letters do they have? What words do they use? 

In [31]:
# Make a list of all bridges, B, and see how many there are
B = [(suf, pre, W.bridges[suf][pre]) for suf in W.bridges for pre in W.bridges[suf]]
len(B)

56477

In [32]:
B[::2000] # Sample every 2000th bridge

[('s', 'laty', (0, (1, 'slaty'))),
 ('p', 'ram', (0, (1, 'pram'))),
 ('a', 'lpha', (0, (1, 'alpha'))),
 ('d', 'ces', (1, (1, 'duces'))),
 ('h', 'uts', (0, (1, 'huts'))),
 ('ke', 'ab', (1, (2, 'kebab'))),
 ('f', 'izz', (0, (1, 'fizz'))),
 ('ho', 'wls', (0, (2, 'howls'))),
 ('c', 'lo', (2, (1, 'cello'))),
 ('g', 'ogo', (0, (1, 'gogo'))),
 ('l', 'th', (1, (1, 'loth'))),
 ('b', 'ola', (0, (1, 'bola'))),
 ('ne', 'ro', (1, (2, 'negro'))),
 ('riv', 'n', (1, (3, 'riven'))),
 ('li', 'szt', (0, (2, 'liszt'))),
 ('on', 'ces', (0, (2, 'onces'))),
 ('na', 'l', (1, (2, 'nail'))),
 ('ov', 'um', (0, (2, 'ovum'))),
 ('br', 'ke', (1, (2, 'broke'))),
 ('sti', 'le', (0, (3, 'stile'))),
 ('ax', 'els', (0, (2, 'axels'))),
 ('yea', 'n', (1, (3, 'yearn'))),
 ('whel', 'p', (0, (4, 'whelp'))),
 ('cabe', 'r', (0, (4, 'caber'))),
 ('fal', 'ls', (0, (3, 'falls'))),
 ('cza', 'r', (0, (3, 'czar'))),
 ('snuc', 'k', (0, (4, 'snuck'))),
 ('scen', 'e', (0, (4, 'scene'))),
 ('apne', 'a', (0, (4, 'apnea')))]

In [33]:
# Counter of bridge excess letters
Counter(x for (_, _, (x, *_)) in B)

Counter({0: 37189, 1: 16708, 2: 2425, 3: 95, 4: 32, 5: 21, 6: 6, 8: 1})

In [34]:
def average(counter):
    return sum(x * counter[x] for x in counter) / sum(counter.values())

average(_) # Average excess across all bridges

0.3916638631655364

In [35]:
# How many 1-step and 2-step bridges are there?
Counter(len(steps) for (_, _, (_, *steps)) in B)

Counter({1: 56327, 2: 150})

**My fifth question**: What strange letter combinations are there? Let's look at two-letter suffixes or prefixes that only appear in one or two nonsubwords. 

In [36]:
{pre: W.startswith[pre] # Rare two-letter prefixes
 for pre in W.startswith if len(pre) == 2 and len(W.startswith[pre]) in (1, 2)}

{'jn': {'jnanas'},
 'dv': {'dvorak'},
 'if': {'iffiness'},
 'ym': {'ymca'},
 'kw': {'kwachas', 'kwashiorkor'},
 'fj': {'fjords'},
 'ek': {'ekistics'},
 'aj': {'ajar'},
 'xi': {'xiphoids', 'xiphosuran'},
 'sf': {'sforzatos'},
 'yc': {'ycleped', 'yclept'},
 'hd': {'hdqrs'},
 'dn': {'dnieper'},
 'ip': {'ipecacs'},
 'ee': {'eelgrasses', 'eelworm'},
 'qa': {'qaids', 'qatar'},
 'ie': {'ieee'},
 'oj': {'ojibwas'},
 'pf': {'pfennigs'},
 'wu': {'wurzel'},
 'uf': {'ufos'},
 'ik': {'ikebanas', 'ikons'},
 'tc': {'tchaikovsky'},
 'bw': {'bwanas'},
 'zw': {'zwiebacks'},
 'gj': {'gjetosts'},
 'iv': {'ivories', 'ivory'},
 'xm': {'xmases'},
 'zl': {'zlotys'},
 'll': {'llamas', 'llanos'},
 'ct': {'ctrl'},
 'qo': {'qophs'},
 'gw': {'gweducks', 'gweducs'},
 'ez': {'ezekiel'},
 'mc': {'mcdonald'},
 'ay': {'ayahs', 'ayatollahs'},
 'fb': {'fbi'}}

In [37]:
endswith = multimap((w[-2:], w) for w in W.unused)

{suf: endswith[suf] # Rare two-letter suffixes
 for suf in endswith if len(endswith[suf]) in (1, 2)}

{'lm': {'stockholm', 'unhelm'},
 'yx': {'styx'},
 'ao': {'chiao', 'ciao'},
 'oe': {'monroe'},
 'oi': {'hanoi', 'polloi'},
 'tl': {'peyotl', 'shtetl'},
 'nx': {'bronx', 'meninx'},
 'rf': {'waldorf', 'windsurf'},
 'wa': {'kiowa', 'okinawa'},
 'lu': {'honolulu'},
 'ho': {'groucho'},
 'bm': {'ibm', 'icbm'},
 'vo': {'concavo'},
 'zo': {'diazo', 'palazzo'},
 'ud': {'aloud', 'overproud'},
 'pa': {'tampa'},
 'xo': {'convexo'},
 'hr': {'kieselguhr'},
 'hm': {'microhm'},
 'ef': {'unicef'},
 'rb': {'cowherb'},
 'ji': {'fiji'},
 'ep': {'asleep', 'shlep'},
 'td': {'retd'},
 'po': {'troppo'},
 'gm': {'apophthegm'},
 'ub': {'beelzebub'},
 'ku': {'haiku'},
 'hu': {'buchu'},
 'xe': {'deluxe', 'maxixe'},
 'gn': {'champaign'},
 'ug': {'bedrug', 'sparkplug'},
 'ec': {'filespec', 'quebec'},
 'nu': {'vishnu'},
 'ru': {'nehru'},
 'mb': {'clomb', 'whitecomb'},
 'ui': {'maqui', 'prosequi'},
 'sr': {'ussr'},
 'ln': {'lincoln'},
 'xs': {'duplexs'},
 'mp': {'prestamp'},
 'ab': {'skylab'},
 'hn': {'mendelssohn'},


The two-letter prefixes definitely include some strange words.

The list of two-letter suffixes is mostly pointing out flaws in the word list. For example, lots of words end in `ab`: blab, cab, jab, lab, etc. But must of them are subwords (of blabs, cabs, jabs, labs, etc.); only `skylab` made it into the word list in singular form but not plural.

# Comparison to Tom Murphy's Program

To compare my [program](portman.py) to [Murphy's](https://sourceforge.net/p/tom7misc/svn/HEAD/tree/trunk/portmantout/): I used a greedy approach that incrementally builds up a single long portmanteau, extending it via a bridge when necessary. Murphy first built a pool of smaller portmanteaux, then joined them all together. I'm reminded of the [Traveling Salesperson Problem](TSP.ipynb) where one algorithm is to form a single path, always progressing to the nearest neighbor, and another algorithm is to maintain a pool of shorter segments and repeatedly join together the two closest segments. The two approaches are different, but it is not clear whether one is better than the other. You could try it!

(*English trivia:*  my program builds a single path of words, and when the path gets stuck and I need something to allow me to continue, it makes sense to call that thing a **bridge**.  Murphy's program starts by building a large pool of small portmanteaux that he calls **particles**, and when he can build no more particles, his next step is to put two particles together, so he calls it a **join**. The different metaphors for what our programs are doing lead to different terminology for the same idea.)

In terms of implementation, mine is in Python and is concise (139 lines); Murphy's is in C++ and is verbose (1867 lines), although Murphy's code does a lot of extra work that mine doesn't: generating diagrams and animations, and running multiple threads in parallel to implement the random restart idea. 

It appears Murphy didn't quite have the complete concept of **subwords**. He did mention that when he adds `'bulleting'`, he crosses `'bullet'` and `'bulletin'` off the list, but somehow  [his string](http://tom7.org/portmantout/murphy2015portmantout.pdf) contains both `'spectacular'` and `'spectaculars'` in two different places. My guess is that when he adds `'spectaculars'` he crosses off `'spectacular'`, but if he happens to add `'spectacular'` first, he will later add `'spectaculars'`. Support for this view is that his output in `bench.txt` says "I skipped 24319 words that were already substrs", but I computed that there are 44,320 such subwords; he found about half of them. I think those missing 20,001 words are the main reason why my strings are coming in at around 554,000 letters, about 57,000 letters shorter than Murphy's 611,820 letters.

Also, Murphy's joins are always between one-letter prefixes and suffixes. I do the same thing for two-word bridges, because having a `W.bridges[A][B]` for every letter `A` and `B` is  the easiest way to prove that the program will terminate. But for one-word bridges, I allow prefixes and suffixes of any length up to a total of 6 for `len(pre) + len(suf)`. I can get away with this because I limited my candidate pool to the 10,000 `shortwords`. It would have been untenable to build all bridges for all 100,000 words, and probably would not have helped shorten $S$ appreciably.

*Note 2:* I should say that I stole one important trick from Murphy. I started watching his highly-entertaining [video](https://www.youtube.com/watch?time_continue=1&v=QVn2PZGZxaI), but then I paused it because I wanted the fun of solving the problem mostly on my own. After I finished the first version of my program, I returned to the video and [paper](http://tom7.org/portmantout/murphy2015portmantout.pdf) and I noticed that I had a problem in my use of bridges. My program originally looked something like this: 

    (overlap, word) = unused_step(...) or one_word_bridge(...) or two_word_bridge(...)
    
That is, I only considered two-word bridges when there was no one-word bridge, on the theory that one word is shorter than two. But Murphy showed that my theory was wrong: I had `bridges['w']['c'] = 'workaholic'`, a one-word bridge, but he had the two-word bridge  `'war' + 'arc' = 'warc'`, which saves six letters over my single word. After seeing that, I shamelessly copied his approach, and now I too get a four-letter bridge for `'w' + 'c'` (sometimes  `'warc'` and sometimes `'we' + 'etc' = 'wetc'`).

In [38]:
W.bridges['w']['c']

(2, (1, 'war'), (2, 'arc'))

# Conclusion

I'll stop here, but you should feel free to do more experimentation of your own. 

Here are some things you could do to make the portmantouts more interesting:

- Use linguistic resources (such as [pretrained word embeddings](https://nlp.stanford.edu/projects/glove/)) to teach your program what words are related to each other. Encourage the program to place  related words next to each other.
- Use linguistic resources (such as [NLTK](https://github.com/nltk/)) to teach your program where syllable breaks are in words, and what each syllable sounds like. Encourage the program to make overlaps match syllables. (That's why "preferendumdums" sounds better than "fortyphonshore".)

Here are some things you could do to make $S$ shorter:

- **Lookahead**: Unused words are chosen based on the degree of overlap, but nothing else. It might help to prefer unused words which have a suffix that matches the prefix of another unused word. A single-word lookahead or a bbeam search could be used.

- **Reserving words**: It seems like `haydn` and `dnieper` are made to go together; they're the only words with `dn` as an affix. If `haydn` was selected first, `unused_step` would select `dnieper` next, but if `dnieper` as selected first, we'd lose the chance to take advantage of that overlap. Maybe there could be a system that assures `haydn` comes first, or a preprocessing step that joined together words that uniquely go together. This is getting close to what Murphy did in his program.

- **Word choice ordering**: Perhaps `compute_startswith` could sort the words in each key's bucket so that the "difficult" words (say, the ones that end in unusual letters) are encountered earlier in the program's execution, when there are more available words for them to connect to.
  
Here are some things you could do to make the program more robust:

- Write and run unit tests.

- Find other word lists and try the program on them.

- Consider what to do for a wordset that has missing bridges. You could try three-word bridges, you could allow the program to back up and remove a previously-placed word; you could allow the addition of words to the start as well as the end of `P`).